# Asymptotic $\omega$-primality

we load the libraries and tools we need for our program

### Libraries

In [1]:
import cdd

In [2]:
import ImportFromBitbucket
ImportFromBitbucket.loadPyFile('integerSmithNormalFormAndApplications.py')

from  integerSmithNormalFormAndApplications import generatorsToEquations, equationsToGeneratorsHomogeneusCase, sympyMatrix2numpyArray, numpyArray2sympyMatrix, lcm, lcmL

downloading... https://bitbucket.org/juan_ignacio_garcia_garcia/commutativemonoids/raw/master/integerSmithNormalFormAndApplications.py

This is the module integerSmithNormalFormAndApplications. 

For a matrix $A$ with dimesion $(m,n)$ and entries in $\mathbb Z$ the function `integerSmithNormalForm` computes a pair of regular 
squared matrices $R$ ($m x m$) and $C$ ($n x n$) with entries in $\mathbb Z$.

This function is used in the following functions:

- `equationsToGeneratorsHomogeneusCase`: if $5x-7y+3z-2t=0$ and $6x+9y-10z+t=0$ are the defining equations of a subgroup $M$ of $\mathbb Z^4$, the command `equationsToGeneratorsHomogeneusCase(Matrix([[5,-7,3,-2],[6,9,-10,1]]))` returns a matrix whose rows form a minimal system of generator of the subgroup $M$.
- `equationsToGenerators`: if $x-2y+3z+4t=0\mod 2$, $6x+8y-10z-16t=0\mod 3$ and $5x+7y-9z+2t=0$ and the defining equations of a subgroup $M$ of $\mathbb Z^4$, the command `equationsToGenerators(Matrix([[1,-2,3,4],[6,8,-10,-16],[5,

In [3]:
import cdd
import functools
import numpy
import numpy as np
import fractions
import math
import itertools
from numpy import array
from fractions import Fraction
from numpy.random import randint

import doctest
import sympy
from sympy import init_printing, Matrix, pprint
init_printing(use_latex='mathjax')

from IPython.display import Math, display

import kruskalAndLLL
from kruskalAndLLL import LLL_fplll

In [8]:
Fr=numpy.vectorize(Fraction)
dens=numpy.vectorize(lambda x:x.denominator)

In [9]:
ImportFromBitbucket.loadPyFile('minimalsNp.py')

from minimalsNp import *

downloading... https://bitbucket.org/juan_ignacio_garcia_garcia/commutativemonoids/raw/master/minimalsNp.py


### Auxiliary functions

We define the functions we need later

In [10]:
def minInOct(y,debug=False):
    '''Returns a tuple (V,Vaux,mm)
    V are the minimals, Vaux are the whole set of minimals
    V=minimals
    Vaux=the whole output of de libcdd
    mm=maximum of the sum of the coordinates of elements of V
    The parameter y represent the inequalities of the polytope
    '''
    Eq,paux=y
    aux=numpy.concatenate(\
        (numpy.zeros((len(paux),1),dtype=Fraction),numpy.diag(array(paux))),
        axis=1)
    aux=Fr(aux)
    aux=numpy.concatenate((Eq,aux),axis=0)
    if debug:
        print('aux=',aux)
    mat=cdd.Matrix(aux, number_type='fraction')
    if debug:
        print('mat=',mat)
    mat.rep_type = cdd.RepType.INEQUALITY
    poly=cdd.Polyhedron(mat)
    Vaux = poly.get_generators()
    if debug:
        print("Vaux=",Vaux)
    Vaux=array(Vaux)
    V=array(Vaux)
    if debug:
        print("V=",V)
    mm=0
    if len(V)>0:
        V=V[V[:,0]!=0,:]
        V=V[:,1:]
        V=proj(V)
        V=computeMinimals(V)
        #print(V)
        if len(V)>0:
            mm=np.max(np.sum(V,axis=1))
    return (V,Vaux,mm)

Here, we show an example

In [11]:

Eq1=array([[Fraction(-35,1),Fraction(5,1), Fraction(7,1)],
          [Fraction(0,1),Fraction(1,1), Fraction(0,1)],
          [Fraction(0,1),Fraction(0,1), Fraction(1,1)]])


Eq1=Fr([[-5,0,0,2],[0,1,0,0],[0,0,1,0],[0,0,0,1]])


print(np.vectorize(np.int)(Eq1))

mat=cdd.Matrix(Eq1, number_type='fraction')
mat.rep_type = cdd.RepType.INEQUALITY
print('mat=',mat)
poly=cdd.Polyhedron(mat)
Vaux = poly.get_generators()

print(Vaux)

[[-5  0  0  2]
 [ 0  1  0  0]
 [ 0  0  1  0]
 [ 0  0  0  1]]
mat= H-representation
begin
 4 4 rational
 -5 0 0 2
 0 1 0 0
 0 0 1 0
 0 0 0 1
end
V-representation
begin
 4 4 rational
 1 0 0 5/2
 0 0 0 1
 0 1 0 0
 0 0 1 0
end


In [12]:
Eq=array(
        [[Fraction(0, 1), Fraction(1, 1), Fraction(-1, 1), Fraction(0, 1)],
         [Fraction(0, 1), Fraction(1, 1), Fraction(0, 1), Fraction(-1, 1)],
         [Fraction(0, 1), Fraction(-1, 1), Fraction(1, 1), Fraction(0, 1)],
         [Fraction(0, 1), Fraction(-1, 1), Fraction(0, 1), Fraction(1, 1)]] )
print(np.vectorize(np.int)(Eq))
minInOct( ( Eq , (1, 1, -1) ) )

[[ 0  1 -1  0]
 [ 0  1  0 -1]
 [ 0 -1  1  0]
 [ 0 -1  0  1]]


(array([[0, 0, 0]], dtype=object), array([[1, 0, 0, 0]]), 0)

### Main function

In [13]:
def OmegaPrimAsint(gamma,Meq=[],gen=[],debug=False):
    '''Meq is an np.array 2-dim with dtype=np.int
    Meq=array([[5,9],[8,7]],dtype=np.int)
    gamma is an np.array 1-dim with dtype=np.int
    gamma=array([4,1],dtype=np.int)
    OmegaPrimAsint(array([1,1,1],dtype=np.int),array([[1,1,1]],dtype=np.int))
    '''
    if not len(Meq) and len(gen):
        r1,r2=generatorsToEquations(gen)
        r1=sympyMatrix2numpyArray(r1)
        r2=sympyMatrix2numpyArray(r2).T
        r2=(r2==np.zeros_like(r2))
        r2=r2.reshape(r2.shape[1])
        Meq=r1[r2[:],:]
        if debug:
            print('The matrix of homogeneus equations is:\n',Meq)
    
    dim=len(gamma)
    M=array(Fr(Meq),dtype=Fraction)
    pos=list(itertools.product([-1,1],repeat=dim))
    vAux=M.dot(gamma)
    vAux=vAux.reshape(len(vAux),1)

    m1=numpy.concatenate((vAux,-M),axis=1)    
    m2=numpy.concatenate((-vAux,M),axis=1)
    Eq=numpy.concatenate((m1,m2))
    if debug: 
        print("Eq=",Eq,'\n')
    
    aa=[]
    for paux in pos:
        #print('Computing projection:',paux)
        aa.append(minInOct((Eq,paux),False))
        #print()
    #aa=view.map_sync(minInOct,[(Eq,paux) for paux in pos])
    
    maximum1=max([x[2] for x in aa])
    lvAux=[x[1] for x in aa]
    minimalesDePrjs=[x[0] for x in aa]
    
    UnionMinimales=[x for x in minimalesDePrjs if len(x)>0]
    UnionMinimales=functools.reduce(lambda x,y:numpy.concatenate((x,y),axis=0),UnionMinimales)
    
    # remove duplicates
    ConjuntoUnionMinimales=set([tuple(x) for x in UnionMinimales])
    UnionMinimales=array([list(x) for x in ConjuntoUnionMinimales])
    
    MinimalesDeLaUnion=computeMinimals(UnionMinimales)
    
    wpa=numpy.max(numpy.sum(MinimalesDeLaUnion,axis=1))
    wpaUM=numpy.max(numpy.sum(UnionMinimales,axis=1))
    
    #print("-->",UnionMinimales.shape[0])
    
    return (wpaUM,wpa, pos, minimalesDePrjs, lvAux, Eq, UnionMinimales, 
            MinimalesDeLaUnion, maximum1, \
            UnionMinimales.shape[0]-MinimalesDeLaUnion.shape[0],
           [x for x in UnionMinimales.tolist() if x not in MinimalesDeLaUnion.tolist()]
           )

Some examples

In [14]:
OmegaPrimAsint(array([15,0],dtype=np.int),gen=Matrix([[-10,11]]))

(Fraction(33, 2),
 Fraction(33, 2),
 [(-1, -1), (-1, 1), (1, -1), (1, 1)],
 [array([], dtype=float64),
  array([[0, Fraction(33, 2)]], dtype=object),
  array([[15, 0]], dtype=object),
  array([[0, Fraction(33, 2)],
         [15, 0]], dtype=object)],
 [array([], dtype=float64), array([[1, 0, Fraction(33, 2)],
         [0, -1, Fraction(11, 10)]], dtype=object), array([[1, 15, 0],
         [0, 1, Fraction(-11, 10)]], dtype=object), array([[1, 0, Fraction(33, 2)],
         [1, 15, 0]], dtype=object)],
 array([[Fraction(165, 1), Fraction(-11, 1), Fraction(-10, 1)],
        [Fraction(-165, 1), Fraction(11, 1), Fraction(10, 1)]],
       dtype=object),
 array([[0, Fraction(33, 2)],
        [15, 0]], dtype=object),
 array([[0, Fraction(33, 2)],
        [15, 0]], dtype=object),
 Fraction(33, 2),
 0,
 [])

In [15]:
OmegaPrimAsint(array([2,1,1],dtype=np.int),gen=Matrix([[2,-2,2]]))

(5,
 5,
 [(-1, -1, -1),
  (-1, -1, 1),
  (-1, 1, -1),
  (-1, 1, 1),
  (1, -1, -1),
  (1, -1, 1),
  (1, 1, -1),
  (1, 1, 1)],
 [array([], dtype=float64),
  array([], dtype=float64),
  array([[0, 3, 0]], dtype=object),
  array([], dtype=float64),
  array([], dtype=float64),
  array([[3, 0, 2]], dtype=object),
  array([[0, 3, 0],
         [1, 2, 0]], dtype=object),
  array([[1, 2, 0],
         [3, 0, 2]], dtype=object)],
 [array([], dtype=float64), array([], dtype=float64), array([[ 1,  0,  3, -1],
         [ 0, -1,  1, -1]]), array([], dtype=float64), array([], dtype=float64), array([[ 1,  3,  0,  2],
         [ 0,  1, -1,  1]]), array([[ 1,  0,  3, -1],
         [ 1,  1,  2,  0]]), array([[1, 1, 2, 0],
         [1, 3, 0, 2]])],
 array([[Fraction(3, 1), Fraction(-1, 1), Fraction(-1, 1), Fraction(0, 1)],
        [Fraction(-1, 1), Fraction(1, 1), Fraction(0, 1), Fraction(-1, 1)],
        [Fraction(-3, 1), Fraction(1, 1), Fraction(1, 1), Fraction(0, 1)],
        [Fraction(1, 1), Fraction(-1

In [16]:
r=OmegaPrimAsint(array([1,1,1,1],dtype=np.int),gen=Matrix([[3,0,1,-4],[-9,1,0,8]]),debug=True)
r
print(r[-1])
print("---")
for x,y in zip(r[2],r[3]):
    print(x)
    for a in y:
        print(array(a,dtype=float))
        print(a)
    print()

The matrix of homogeneus equations is:
 [[ 1  9 -3  0]
 [ 0 -8  4  1]]
Eq= [[Fraction(7, 1) Fraction(-1, 1) Fraction(-9, 1) Fraction(3, 1)
  Fraction(0, 1)]
 [Fraction(-3, 1) Fraction(0, 1) Fraction(8, 1) Fraction(-4, 1)
  Fraction(-1, 1)]
 [Fraction(-7, 1) Fraction(1, 1) Fraction(9, 1) Fraction(-3, 1)
  Fraction(0, 1)]
 [Fraction(3, 1) Fraction(0, 1) Fraction(-8, 1) Fraction(4, 1)
  Fraction(1, 1)]] 

[[7, 0, 0, 0]]
---
(-1, -1, -1, -1)

(-1, -1, -1, 1)
[0.         0.         0.         6.33333333]
[0 0 0 Fraction(19, 3)]

(-1, -1, 1, -1)

(-1, -1, 1, 1)

(-1, 1, -1, -1)

(-1, 1, -1, 1)
[0.         0.         0.         6.33333333]
[0 0 0 Fraction(19, 3)]
[0.         0.77777778 0.         3.22222222]
[0 Fraction(7, 9) 0 Fraction(29, 9)]

(-1, 1, 1, -1)
[0.         1.58333333 2.41666667 0.        ]
[0 Fraction(19, 12) Fraction(29, 12) 0]

(-1, 1, 1, 1)
[0.         1.58333333 2.41666667 0.        ]
[0 Fraction(19, 12) Fraction(29, 12) 0]
[0.         0.77777778 0.         3.22222222]
[0 

In [17]:
print(r[7])
print()
print(r[6])
print()
for x in r[3]:
    print(x)

[[Fraction(29, 8) Fraction(3, 8) 0 0]
 [0 0 0 Fraction(19, 3)]
 [Fraction(19, 4) 0 0 0]
 [0 Fraction(7, 9) 0 Fraction(29, 9)]
 [0 Fraction(19, 12) Fraction(29, 12) 0]]

[[Fraction(29, 8) Fraction(3, 8) 0 0]
 [7 0 0 0]
 [0 0 0 Fraction(19, 3)]
 [Fraction(19, 4) 0 0 0]
 [0 Fraction(7, 9) 0 Fraction(29, 9)]
 [0 Fraction(19, 12) Fraction(29, 12) 0]]

[]
[[0 0 0 Fraction(19, 3)]]
[]
[]
[]
[[0 0 0 Fraction(19, 3)]
 [0 Fraction(7, 9) 0 Fraction(29, 9)]]
[[0 Fraction(19, 12) Fraction(29, 12) 0]]
[[0 Fraction(19, 12) Fraction(29, 12) 0]
 [0 Fraction(7, 9) 0 Fraction(29, 9)]]
[[Fraction(19, 4) 0 0 0]]
[[0 0 0 Fraction(19, 3)]
 [Fraction(19, 4) 0 0 0]]
[[7 0 0 0]]
[]
[[Fraction(29, 8) Fraction(3, 8) 0 0]
 [Fraction(19, 4) 0 0 0]]
[[0 0 0 Fraction(19, 3)]
 [0 Fraction(7, 9) 0 Fraction(29, 9)]
 [Fraction(29, 8) Fraction(3, 8) 0 0]
 [Fraction(19, 4) 0 0 0]]
[[0 Fraction(19, 12) Fraction(29, 12) 0]
 [Fraction(29, 8) Fraction(3, 8) 0 0]
 [7 0 0 0]]
[[0 Fraction(7, 9) 0 Fraction(29, 9)]
 [0 Fraction(19

If the above function computes the asymptotic $\omega$-primality, then the function below computes the asymptotic $\omega$ primality of a cancellative monoid.

In [17]:
def AsympOmegaPrimOfCancellativeMonoid(Mcad):
    '''Mcad es una lista de cadenas
    Mcad=[['2.3','5.7'],['1.2','3/7']]
    '''
    p=len(Mcad[0])
    return max( [OmegaPrimAsint(omega,Mcad)[0] for  omega in numpy.eye(p)] )

### Example generator

In [18]:
def matrixGenAux(paux,x):
    pass
def preImageInQuadrant(paux,x,Meq,gamma):
    paux=(-1,-1,1)
    x=array([0,0,10])
    gamma=array([1,1,1])
    Meq=array([[1,2,3]])
    cGamma=array([gamma]).T
    vAux=Meq.dot(cGamma)
    m1=numpy.concatenate((vAux,-Meq),axis=1)
    m2=numpy.concatenate((-vAux,Meq),axis=1)
    Eq=numpy.concatenate((m1,m2))
    #Eq,paux=y
    aux=numpy.concatenate( (numpy.zeros((len(paux),1)), numpy.diag(array(paux))), axis=1)
    aux=numpy.concatenate((Eq,aux),axis=0)
    aux=numpy.concatenate( (aux, array([[10,0,0,-1],[-10,0,0,1]]) ), axis=0 )
    print('aux=',aux)
    mat=cdd.Matrix(aux, number_type='fraction')
    mat.rep_type = cdd.RepType.INEQUALITY
    poly=cdd.Polyhedron(mat)
    Vaux = poly.get_generators()
    print(Vaux)

The function below computes examples where the computation of the asymptotic $\omega$ primality is not clear.

In [19]:
def findEx(dim,Total,nEquations,debug=False):
    '''Find examples fulfilling the above property
    dim: dimension 
    Total: total number of random examples
    k: number of additional equations, 0<=k<=dim-2
    '''
    k=nEquations-1
    lMUeqUM=[] # list of booleans
    lWPeqWPProj=[] # list of booleans
    lTodos=[]
    contador=0
    for i in range(Total):
        omega=randint(1,2,(dim))
        rr=True
        while rr:
            epos=randint(1,2,(1,dim))
            eresto=randint(-7,8,(k,dim))
            eqM=numpy.concatenate((epos,eresto),axis=0)
            if np.linalg.matrix_rank(eqM)==eqM.shape[0]:
                rr=False
        # eqM are the equations of M, S=N^dim/~M
        lTodos.append((omega,eqM))
        sal=OmegaPrimAsint(omega,eqM,debug=debug)
        # print(sal[-1],' ',sal[-2],' ',sal[0])
        # sal=(
        # wpa, pos, minimalesDePrjs, lvAux (lista cdd), Eq, UnionMinimales, MinimalesDeLaUnion, 
        # maximum1 (maximo de la union de los minimales), 
        # UnionMinimales.shape[0]-MinimalesDeLaUnion.shape[0] (si es cero no sobran minimales)
        # )
        if sal[-2]!=0:
            lMUeqUM.append(False)
        else:
            lMUeqUM.append(True)
            
        if sal[-3]==sal[1]:
            lWPeqWPProj.append(True)
        else:
            lWPeqWPProj.append(False)
        if sal[-2]!=0 and sal[-3]!=sal[1]:
            contador=contador+1
    print('Number of fails:',contador, '; dim:',str(dim),'; Total:',str(Total),
          '; nEquations:',str(nEquations))
        
    return(lTodos,lMUeqUM,lWPeqWPProj)

In [24]:
findEx(4,10,2,False)

Number of fails: 1 ; dim: 4 ; Total: 10 ; nEquations: 2


([(array([1, 1, 1, 1]), array([[ 1,  1,  1,  1],
          [ 1,  3, -4, -3]])), (array([1, 1, 1, 1]), array([[ 1,  1,  1,  1],
          [ 6,  0, -7,  1]])), (array([1, 1, 1, 1]), array([[ 1,  1,  1,  1],
          [ 4, -4,  1, -2]])), (array([1, 1, 1, 1]), array([[ 1,  1,  1,  1],
          [-6, -1,  0,  2]])), (array([1, 1, 1, 1]), array([[ 1,  1,  1,  1],
          [-1,  4,  1,  5]])), (array([1, 1, 1, 1]), array([[ 1,  1,  1,  1],
          [ 3,  6,  3, -7]])), (array([1, 1, 1, 1]), array([[ 1,  1,  1,  1],
          [ 2, -7, -4, -6]])), (array([1, 1, 1, 1]), array([[ 1,  1,  1,  1],
          [-5, -6,  6, -5]])), (array([1, 1, 1, 1]), array([[ 1,  1,  1,  1],
          [-6,  7,  4,  3]])), (array([1, 1, 1, 1]), array([[ 1,  1,  1,  1],
          [-1, -6, -6,  0]]))],
 [True, True, True, False, True, True, False, True, False, True],
 [True, True, True, True, True, True, True, True, False, True])

In [36]:
sal = OmegaPrimAsint([1, 1, 1, 1], Meq=array([[ 1,  1,  1,  1],         [ -6,  7, 4,  3]]) )
sal[-1]

[[0, 0, 0, 8]]

In [37]:
sal[6]

array([[0, 0, Fraction(20, 3), 0],
       [0, 0, 0, 5],
       [Fraction(4, 5), 0, Fraction(16, 5), 0],
       [Fraction(20, 13), Fraction(32, 13), 0, 0],
       [0, 0, 0, 8],
       [Fraction(4, 9), 0, 0, Fraction(32, 9)]], dtype=object)

In [38]:
sal[7]

array([[0, 0, Fraction(20, 3), 0],
       [0, 0, 0, 5],
       [Fraction(4, 5), 0, Fraction(16, 5), 0],
       [Fraction(20, 13), Fraction(32, 13), 0, 0],
       [Fraction(4, 9), 0, 0, Fraction(32, 9)]], dtype=object)

In [21]:
dim=4
output=[]
for neq in range(2,dim-1):
    output=findEx(dim,10,neq,True)

Eq= [[Fraction(4, 1) Fraction(-1, 1) Fraction(-1, 1) Fraction(-1, 1)
  Fraction(-1, 1)]
 [Fraction(-13, 1) Fraction(7, 1) Fraction(-7, 1) Fraction(7, 1)
  Fraction(6, 1)]
 [Fraction(-4, 1) Fraction(1, 1) Fraction(1, 1) Fraction(1, 1)
  Fraction(1, 1)]
 [Fraction(13, 1) Fraction(-7, 1) Fraction(7, 1) Fraction(-7, 1)
  Fraction(-6, 1)]] 

Eq= [[Fraction(4, 1) Fraction(-1, 1) Fraction(-1, 1) Fraction(-1, 1)
  Fraction(-1, 1)]
 [Fraction(9, 1) Fraction(-6, 1) Fraction(5, 1) Fraction(-5, 1)
  Fraction(-3, 1)]
 [Fraction(-4, 1) Fraction(1, 1) Fraction(1, 1) Fraction(1, 1)
  Fraction(1, 1)]
 [Fraction(-9, 1) Fraction(6, 1) Fraction(-5, 1) Fraction(5, 1)
  Fraction(3, 1)]] 

Eq= [[Fraction(4, 1) Fraction(-1, 1) Fraction(-1, 1) Fraction(-1, 1)
  Fraction(-1, 1)]
 [Fraction(3, 1) Fraction(-5, 1) Fraction(2, 1) Fraction(7, 1)
  Fraction(-7, 1)]
 [Fraction(-4, 1) Fraction(1, 1) Fraction(1, 1) Fraction(1, 1)
  Fraction(1, 1)]
 [Fraction(-3, 1) Fraction(5, 1) Fraction(-2, 1) Fraction(-7, 1)
  Fracti